<a href="https://colab.research.google.com/github/AhmadJamal01/Floodead-Inside/blob/main/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import glob
import os
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from osgeo import gdal
import cv2
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


NUM_FOLDS = 10

## Read the Data

In [ ]:
!pip install gdal > /dev/null

In [ ]:
import gdown
gdown.download("https://drive.google.com/uc?id=1och-QmNa3FAiS-wssgzCwISbmpSezIi_", "dataset.zip", quiet=False)
gdown.extractall("dataset.zip")
path = 'dataset/'


## Prepare the Data

In [ ]:
df = pd.DataFrame(columns=['image_path', 'label'])

for image_path in sorted(glob.glob('dataset/flooded/*.jpg')):
    data = {'image_path': image_path, 'label': 'flooded'}
    df.loc[len(df)] = data

for image_path in sorted(glob.glob('dataset/non-flooded/*.jpg')):
    data = {'image_path': image_path, 'label': 'non-flooded'}
    df.loc[len(df)] = data

In [ ]:
print(df.head())
print("Dataset shape:", df.shape)

### Extract Features

In [ ]:
def extract_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features = hog(gray_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
    return hog_features


In [ ]:
hog_features = []
labels = []

for index, row in df.iterrows():
    image_path = row['image_path']
    label = row['label']
    
    # Read the image
    image = cv2.imread(image_path)
    
    # Extract HOG features
    hog_feature = extract_hog_features(image)
    
    # Append the features and labels to the respective lists
    hog_features.append(hog_feature)
    labels.append(label)


## Split the Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2, random_state=42)


## Model

In [ ]:
svm = LinearSVC()
svm.fit(X_train, y_train)


## Evaluation

In [ ]:
y_pred = svm.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}%'.format(accuracy * 100))
